Các thư viện được sử dụng

In [1]:
from pysat.solvers import Glucose3
from itertools import combinations
import itertools

Tạo lớp MatrixColoring gồm các phương thức: 
+ get_var_id 
+ get_adjacent_variables 
+ generate_cnf_clauses 
+ solve 
+ from_file 
+ print_solution



In [2]:
class MatrixColoring:
    def __init__(self, n, m, nonogram):
        self.n = n
        self.m = m
        self.nonogram = nonogram

    # Được sử dụng để tính toán mã số biến đại diện cho một ô trong bảng.
    def get_var_id(self, i, j):
        return i * self.m + j + 1

    # Được sử dụng để lấy danh sách các biến đại diện cho các ô kề với một ô cụ thể trong bảng.
    def get_adjacent_variables(self, i, j):
        neighbors = []
        # Duyệt các ô trong ma trận
        for x in range(i - 1, i + 2):
            for y in range(j - 1, j + 2):
                # Nếu ô đó nằm ngoài kích thước bản thì bỏ qua
                if x < 0 or x >= self.n or y < 0 or y >= self.m:
                    continue
                # Nếu ô nằm trong bảng, tính toán mã số biến đại diện và thêm vào danh sách neighbors
                neighbors.append(self.get_var_id(x, y))
        # Trả về danh sách các biến đại diện cho ô kề với ô (i, j)
        return neighbors
    
    # Tạo ra các mệnh đề CNF (Conjunctive Normal Form) từ ma trận nonogram (biểu diễn cho bài toán coloring) và lưu chúng vào danh sách clauses.
    def generate_cnf_clauses(self):
        clauses = []
        for i in range(self.n):
            for j in range(self.m):
                # Với mỗi ô trong ma trận, tạo ra một biến đại diện (dưới dạng số nguyên dương) và thêm hai mệnh đề vào clauses là var_id và -var_id.
                var_id = self.get_var_id(i, j)
                clauses.append([var_id, -var_id])
                cell_value = self.nonogram[i][j]
                # Nếu ô đó có giá trị là 0 (ô trống), thêm vào clauses một mệnh đề có chứa bất kỳ biến nào trong các ô xung quanh ô đó.
                if isinstance(cell_value, int) and cell_value == 0:
                    neighbors = self.get_adjacent_variables(i, j)
                    for neighbor in neighbors:
                        clauses.append([-neighbor])
                # Nếu ô đó có giá trị lớn hơn 0 (ô được sơn), tạo tất cả các tổ hợp có cell_value biến trong số các ô xung quanh ô đó.
                if isinstance(cell_value, int) and cell_value > 0:
                    # Lấy tất cả các tổ hợp (hay "combinations") của các ô kề cạnh này với độ dài bằng cell_value
                    neighbors = self.get_adjacent_variables(i, j)
                    for combo in combinations(neighbors, cell_value):
                        # Đối với mỗi tổ hợp này, chương trình sẽ tạo ra một danh sách (hay "combo_list") và tạo ra một danh sách các ô kề cạnh còn lại (hay "remaining_neighbors") bằng cách loại bỏ các ô trong danh sách combo_list.
                        combo_list = list(combo)
                        remaining_neighbors = [n for n in neighbors if n not in combo_list]
                        # Đối với mỗi ô trong danh sách combo_list, chương trình sẽ tạo ra một mệnh đề logic bằng cách kết hợp tất cả các ô còn lại trong danh sách remaining_neighbors với ô này.
                        for x in combo_list:
                            clauses.append(remaining_neighbors + [x])
                        # Đối với mỗi ô trong danh sách remaining_neighbors, chương trình sẽ tạo ra một mệnh đề logic bằng cách kết hợp tất cả các ô trong danh sách combo_list với ô này.
                        for y in remaining_neighbors:
                            temp = [-v for v in combo_list] + [-y]
                            clauses.append(temp)
        return clauses

    # Tạo ra danh sách các mệnh đề CNF bằng cách gọi hàm generate_cnf_clauses, sau đó sử dụng bộ giải Glucose3 để giải quyết bài toán.
    def solve(self):
        clauses = self.generate_cnf_clauses()
        g = Glucose3()

        # Thêm các mệnh đề CNF vào Glucose3
        for clause in clauses:
            g.add_clause(clause)

        # Giải quyết bài toán và trả về kết quả
        if g.solve():
            model = g.get_model()
            return [[True if model[i * self.m + j] > 0 else False for j in range(self.m)] for i in range(self.n)]
        else:
            return None

    # Được dùng để tạo một đối tượng Nonogram từ một file text chứa thông tin về nonogram.
    @classmethod
    def from_file(cls, filename):
        with open(filename, 'r') as f:
            lines = f.readlines()
            m, n = map(int, lines[0].strip().split())
            nonogram = []
            for line in lines[1:]:
                row = [int(c) if c != '.' else c for c in line.strip().split()]
                nonogram.append(row)
        return cls(n, m, nonogram)
    
    # Dùng để hiển thị kết quả của bài toán Nonogram.
    def print_solution(self, solution):
        if solution:
            for x, row in enumerate(solution):
                for y, value in enumerate(row):
                    char = ' ' if self.nonogram[x][y] == '.' else str(self.nonogram[x][y])
                    color = '\033[42m' if value else '\033[41m'
                    reset = '\033[0m'
                    print(f"{color}{char}{reset}", end='')
                print()
        else:
            print("No solution found")

Thực thi code

In [3]:
solver = MatrixColoring.from_file('input2.txt')
solver = MatrixColoring.from_file('input.txt')
result = solver.solve()
solver.print_solution(result)

 23  0    
    3 2  6
  5 53 574
 4 5 5 6 3
  4 5 6  3
   2 5    
4 1   11  
4 1   1 4 
    6    4
 44    4  
